# <center> 👉 class_10_1 Video » _Realtime Canny Edge, Road Lane Detection_ </center>

## Canny Edge Detection

In [4]:
import cv2  
import numpy as np  
  
cap = cv2.VideoCapture(0)  
  
while (1):  
  
    ret, frame = cap.read()  
    cv2.imshow('Original', frame)  
    edges = cv2.Canny(frame, 100, 200,True)  
    cv2.imshow('Edges', edges)  
  
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()  
cv2.destroyAllWindows()  

## Road Lane Detection

### Steps in Road Lane Detection  

1. Capturing and decoding video file: 
>- We will capture the video using VideoFileClip object and after the capturing has been initialized every video frame is decoded (i.e. converting into a sequence of images).
2. Grayscale conversion of image: 
>- The video frames are in RGB format, RGB is converted to grayscale because processing a single channel image is faster than processing a three-channel colored image.
3. Reduce noise: 
>- Noise can create false edges, therefore before going further, it’s imperative to perform image smoothening.   
>- Gaussian blur is used to perform this process. Gaussian blur is a typical image filtering technique for lowering noise and enhancing image characteristics.   
>- The weights are selected using a Gaussian distribution, and each pixel is subjected to a weighted average that considers the pixels surrounding it.   
>- By reducing high-frequency elements and improving overall image quality, this blurring technique creates softer, more visually pleasant images.  
4. Canny Edge Detector: 
>- It computes gradient in all directions of our blurred image and traces the edges with large changes in intensity.   
5. Region of Interest: 
>- This step is to take into account only the region covered by the road lane.  
>- A mask is created here, which is of the same dimension as our road image.   
>- Furthermore, bitwise AND operation is performed between each pixel of our canny image and this mask.   
>- It ultimately masks the canny image and shows the region of interest traced by the polygonal contour of the mask.
6. Hough Line Transform: 
>- In image processing, the Hough transformation is a feature extraction method used to find basic geometric objects like lines and circles.   
>- By converting the picture space into a parameter space, it makes it possible to identify shapes by accumulating voting points.   
>- We'll use the probabilistic Hough Line Transform in our algorithm.   
>- The Hough transformation has been extended to address the computational complexity with the probabilistic Hough transformation.   
>- In order to speed up processing while preserving accuracy in shape detection, it randomly chooses a selection of picture points and applies the Hough transformation solely to those points.
7. Draw lines on the Image or Video:  
>- After identifying lane lines in our field of interest using Hough Line Transform, we overlay them on our visual input(video stream/image).


In [7]:
import numpy as np
import pandas as pd
import cv2

def process_video(test_video, output_video):
    """
    Read input video stream and produce a video file with detected lane lines.
    Parameters:
        test_video: location of input video file
        output_video: location where output video file is to be saved  
    """
    input_video = editor.VideoFileClip(test_video, audio=False)  # read the video file using VideoFileClip without audio
    processed = input_video.fl_image(frame_processor)
    processed.write_videofile(output_video, audio=False)   # save the output video stream to an mp4 file

def frame_processor(image):
    """
    Process the input frame to detect lane lines.
    Parameters:
        image: image of a road where one wants to detect lane lines
        (we will be passing frames of video to this function)
    """
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel_size = 5
    blur = cv2.GaussianBlur(grayscale, (kernel_size, kernel_size), 0)
    low_t = 50                             # first threshold for the hysteresis procedure
    high_t = 150                           # second threshold for the hysteresis procedure 
    edges = cv2.Canny(blur, low_t, high_t) # applying canny edge detection and save edges in a variable
    region = region_selection(edges)       # since we are getting too many edges from our image, we apply a mask polygon to only focus on the road
    hough = hough_transform(region)        # Applying hough transform 
    result = draw_lane_lines(image, lane_lines(image, hough))  # draw the lines
    return result

def region_selection(image):
    """
    Determine and cut the region of interest in the input image.
    Parameters:
        image: we pass here the output from canny where we have 
        identified edges in the frame
    """
    
    mask = np.zeros_like(image)    # create an array of the same size as of the input image 
    
    if len(image.shape) > 2:       # if you pass an image with more then one channel
        channel_count = image.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:                          # this image only has one channel 
        ignore_mask_color = 255    # color of the mask polygon (white)
    
    rows, cols = image.shape[:2]              # creating a polygon to focus only on the road in the picture
    bottom_left  = [cols * 0.1, rows * 0.95]  # create polygon (in accordance to how the camera was placed)
    top_left     = [cols * 0.4, rows * 0.6]
    bottom_right = [cols * 0.9, rows * 0.95]
    top_right    = [cols * 0.6, rows * 0.6]
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)  # filling the polygon with white color and generating the final mask
    masked_image = cv2.bitwise_and(image, mask)   # Bitwise AND on the input image and mask to get only the edges on the road
    return masked_image  

def hough_transform(image):
    """
    Determine and cut the region of interest in the input image.
    Parameter:
        image: grayscale image which should be an output from the edge detector
    """
    rho = 1              # Distance resolution of the accumulator in pixels                    
    theta = np.pi/180    # Angle resolution of the accumulator in radians.
    threshold = 20       # Only lines that are greater than threshold will be returned.
    minLineLength = 20   # Line segments shorter than that are rejected.
    maxLineGap = 500     # Maximum allowed gap between points on the same line to link them  
    
    return cv2.HoughLinesP(image, rho = rho, theta = theta, threshold = threshold, minLineLength = minLineLength, maxLineGap = maxLineGap)
                         # returns an array containing dimensions of straight lines 

def average_slope_intercept(lines):
    """
    Find the slope and intercept of the left and right lanes of each image.
    Parameters:
        lines: output from Hough Transform
    """
    left_lines    = []              #(slope, intercept)
    left_weights  = []              #(length,)
    right_lines   = []              #(slope, intercept)
    right_weights = []              #(length,)
     
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                continue
            slope = (y2 - y1) / (x2 - x1)     # calculating slope of a line
            intercept = y1 - (slope * x1)     # calculating intercept of a line
            length = np.sqrt(((y2 - y1) ** 2) + ((x2 - x1) ** 2))  # calculating length of a line
            
            if slope < 0:     # slope of left lane is negative and for right lane slope is positive
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
                
    left_lane  = np.dot(left_weights,  left_lines) / np.sum(left_weights)  if len(left_weights) > 0 else None
    right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None
    return left_lane, right_lane
   
def pixel_points(y1, y2, line):
    """
    Converts the slope and intercept of each line into pixel points.
        Parameters:
            y1: y-value of the line's starting point.
            y2: y-value of the line's end point.
            line: The slope and intercept of the line.
    """
    if line is None:
        return None
    slope, intercept = line
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    return ((x1, y1), (x2, y2))
   
def lane_lines(image, lines):
    """
    Create full lenght lines from pixel points.
        Parameters:
            image: The input test image.
            lines: The output lines from Hough Transform.
    """
    left_lane, right_lane = average_slope_intercept(lines)
    y1 = image.shape[0]
    y2 = y1 * 0.6
    left_line  = pixel_points(y1, y2, left_lane)
    right_line = pixel_points(y1, y2, right_lane)
    return left_line, right_line
 
def draw_lane_lines(image, lines, color=[255, 0, 0], thickness=12):
    """
    Draw lines onto the input image.
        Parameters:
            image: The input test image (video frame in our case).
            lines: The output lines from Hough Transform.
            color (Default = red): Line color.
            thickness (Default = 12): Line thickness. 
    """
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_image, *line,  color, thickness)
    return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)

cap = cv2.VideoCapture('./Videos/lane.mp4')  
  
while (1):  
  
    ret, frame = cap.read()  
    cv2.imshow('Original', frame)  
    result=frame_processor(frame)
    cv2.imshow('Lanes', result)  
  
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()  
cv2.destroyAllWindows()  